Alumna : Ju'arez Torres Karla Romina

Profesor : Francisco Pérez Carbajal

Materia : Proyecto 1


In [1]:
pip install flexpolyline

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Biblioteca para realizar solicitudes HTTP
import requests

# Manipulación de datos
import pandas as pd

# Imprime estructuras de datos complejas de manera legible
import pprint

# Crea mapas interactivos
import folium

# Decodifica polilíneas en formato FlexPolyline
import flexpolyline as fp

# Visualización de datos estadísticos
import seaborn as sns

# Tiempo
import pytz
from datetime import datetime

# Manipulación de datos numéricos
import numpy as np

# Visualiza las rutas
import IPython
# Random
import random

Matplotlib is building the font cache; this may take a moment.


In [2]:
def utctodate(utc):# Obtener la zona horaria local
    local_tz = pytz.timezone('America/Mexico_City')

    # Convertir el timestamp a datetime
    utc_time = datetime.fromtimestamp(utc, pytz.utc)

    # Convertir de UTC a la zona horaria local
    local_time = utc_time.astimezone(local_tz)

    # Formatear la fecha y hora
    formatted_time = local_time.strftime('%Y-%m-%d %H:%M:%S')
    return formatted_time

In [3]:
URL = 'https://gbfs.mex.lyftbikes.com/gbfs/en/station_information.json'
r = requests.get(URL)

In [ ]:
pprint.pprint(r.json())

In [ ]:
# Define la URL donde se encuentra el archivo JSON con la información
# de las estaciones de bicicletas
URL = 'https://gbfs.mex.lyftbikes.com/gbfs/en/station_information.json'

# Realiza una solicitud GET a la URL en formato JSON
r = requests.get(URL)

# Convierte el contenido de la respuesta a un diccionario de Python y
# extrae la información de las estaciones
stations_data = r.json()['data']['stations']

# Convierte los datos de las estaciones en un DataFrame de Pandas
df_stations = pd.DataFrame(stations_data)

#df_stations = df_stations[df_stations['lat']!=0]
df_stations_station_id = df_stations.station_id
df_stations_short_name = df_stations.short_name
df_stations_lat = df_stations.lat
df_stations_lon = df_stations.lon
df_stations_capacity = df_stations.capacity

df_part1 = pd.concat([df_stations_station_id,df_stations_short_name,df_stations_lat,df_stations_lon,df_stations_capacity],axis=1)

In [ ]:
URL = 'https://gbfs.mex.lyftbikes.com/gbfs/en/station_status.json'
r = requests.get(URL)
df_status_stations = pd.DataFrame(r.json()['data']['stations'])
#df_status_stations = df_status_stations[df_status_stations['is_installed']!=0]

In [ ]:
df_status_stations_num_bikes_available = df_status_stations.num_bikes_available
df_status_stations_num_bikes_disabled = df_status_stations.num_bikes_disabled
df_status_stations_num_docks_available = df_status_stations.num_docks_available
df_status_stations_num_docks_disabled = df_status_stations.num_docks_disabled

df_part2 = pd.concat([df_status_stations_num_bikes_available,df_status_stations_num_bikes_disabled,df_status_stations_num_docks_available,df_status_stations_num_docks_disabled],axis=1)

In [ ]:
last_updated_list = []
utclu = utctodate(r.json()['last_updated'])
for i in range(len(df_status_stations)):
    last_updated_list.append(utclu)
df_part3 = pd.DataFrame({'last_updated': last_updated_list})

In [ ]:
print('723057d487f95517ccc17f5f7d6d2043')
API_KEY = input("Please enter your OpenWeatherMap API key: ")#
#API_KEY = 'e34c77410f818e358970a07534bb6afa'

In [ ]:
temp_c_list = []
wind_speed_list = []
humidity_list = []
clouds_list = []

In [ ]:
i = 0
URLWM_base = 'https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&units=metric&appid={API}'

for i in range(len(df_stations_lat)):
#for i in range(5):
    lati = df_stations_lat[i]
    longi = df_stations_lon[i]
    print('Datos obtenidos para:' ,lati, longi)
    URLWM = URLWM_base.format(lat=lati, lon=longi, API=API_KEY)
    print(URLWM)
    rWM = requests.get(URLWM)
    temp_c_list.append(rWM.json()['main']['temp'])
    wind_speed_list.append(rWM.json()['wind']['speed'])
    humidity_list.append(rWM.json()['main']['humidity'])
    clouds_list.append(rWM.json()['clouds']['all'])
    print(URLWM)

In [ ]:
len(temp_c_list)

In [ ]:
print(temp_c_list)
print(wind_speed_list)
print(humidity_list)
print(clouds_list)

In [ ]:
df_temp_c= pd.DataFrame({'temp_c': temp_c_list})
df_wind_kph = pd.DataFrame({'wind_kph': wind_speed_list})
df_humidity = pd.DataFrame({'humidity': humidity_list})
df_cloud = pd.DataFrame({'clouds': clouds_list})

In [ ]:
df_part4 = pd.concat([df_temp_c,df_wind_kph,df_humidity,df_cloud],axis=1)
df_part4

In [ ]:
Dataframe_final = pd.concat([df_part1,df_part2,df_part3,df_part4],axis=1)
Dataframe_final = Dataframe_final[Dataframe_final['lat']!=0]
#df_status_stations = df_status_stations[df_status_stations['is_installed']!=0]

In [ ]:
Dataframe_final.to_csv('Dataframe_final.csv')

In [ ]:
avg_temp = Dataframe_final['temp_c'].mean()
print(f"La temperatura promedio en todas las estaciones es de: {avg_temp:.2f} degrees Celsius")


In [ ]:
sns.boxplot(data=Dataframe_final[['num_bikes_available', 'num_bikes_disabled', 'num_docks_available', 'num_docks_disabled']])


In [ ]:
# Análisis de los boxplots

# Mediana:
# - num_bikes_available: La mediana se encuentra alrededor de 3 bicicletas disponibles.
# - num_bikes_disabled: La mediana indica que típicamente hay 0 bicicletas deshabilitadas.
# - num_docks_available: La mediana muestra alrededor de 7 espacios disponibles.
# - num_docks_disabled: La mediana sugiere que usualmente hay 0 espacios deshabilitados.

# Dispersión:
# - num_bikes_available: Los datos muestran una dispersión considerable, con un rango intercuartil amplio. Esto indica una variabilidad en la cantidad de bicicletas disponibles en las estaciones.
# - num_bikes_disabled: La dispersión es menor, concentrándose en valores bajos.
# - num_docks_available: La dispersión es moderada, con un rango intercuartil más estrecho que  el de bicicletas disponibles.
# - num_docks_disabled: La dispersión es mínima, similar a la de bicicletas deshabilitadas.

# Valores atípicos:
# - num_bikes_available: Se observan algunos valores atípicos superiores, indicando estaciones con una cantidad significativamente mayor de bicicletas disponibles en comparación con la mayoría.
# - num_docks_available: También se presentan valores atípicos superiores, representando estaciones con una mayor cantidad de espacios disponibles.

# Conclusiones:
# - En general, las estaciones tienden a tener una cantidad moderada de bicicletas disponibles y espacios libres.
# - La variabilidad en la cantidad de bicicletas disponibles sugiere que algunas estaciones experimentan una mayor demanda o reabastecimiento.
# - La presencia de valores atípicos indica que existen estaciones con características distintas en términos de disponibilidad de bicicletas y espacios.


In [ ]:
#Determina entre todas las estaciones qué estaciones están más cerca entre sí
# geográficamente.

from sklearn.neighbors import BallTree

# Convertir las coordenadas de latitud y longitud a radianes
Dataframe_final['lat_rad'] = np.radians(Dataframe_final['lat'])
Dataframe_final['lon_rad'] = np.radians(Dataframe_final['lon'])

# Crear una matriz de coordenadas
coords = Dataframe_final[['lat_rad', 'lon_rad']].values

# Crear un BallTree
tree = BallTree(coords, metric='haversine')

# Encontrar los vecinos más cercanos para cada punto
distances, indices = tree.query(coords, k=2)

# La distancia al vecino más cercano está en la segunda columna (índice 1)
# ya que la primera columna (índice 0) es la distancia a sí mismo.
Dataframe_final['nearest_distance'] = distances[:, 1]
Dataframe_final['nearest_station'] = indices[:, 1]

# Convertir la distancia de radianes a kilómetros
earth_radius = 6371
Dataframe_final['nearest_distance'] = Dataframe_final['nearest_distance'] * earth_radius

# Encontrar las dos estaciones más cercanas entre sí
min_distance = Dataframe_final['nearest_distance'].min()
closest_stations = Dataframe_final[Dataframe_final['nearest_distance'] == min_distance]

# Imprimir los resultados
print(f"Las dos estaciones más cercanas están a una distancia de {min_distance:.2f} km.")
print(closest_stations[['short_name', 'nearest_distance', 'nearest_station']])

# Obtener los nombres de las estaciones más cercanas
station1_index = closest_stations.index[0]
station2_index = closest_stations['nearest_station'].iloc[0]
station1_name = Dataframe_final['short_name'][station1_index]
station2_name = Dataframe_final['short_name'][station2_index]

print(f"Las estaciones más cercanas son: {station1_name} y {station2_name}")


In [ ]:
#Determina cual es la estación más al oeste y la que está más al este

# Encuentra la estación más al oeste (menor longitud)
west_station = Dataframe_final[Dataframe_final['lon'] == Dataframe_final['lon'].min()]

# Obtener la latitud y longitud de la estación más al oeste
west_lat = west_station['lat'].values[0]
west_lon = west_station['lon'].values[0]


# Encuentra la estación más al este (mayor longitud)
east_station = Dataframe_final[Dataframe_final['lon'] == Dataframe_final['lon'].max()]


# Obtener la latitud y longitud de la estación más al este
east_lat = east_station['lat'].values[0]
east_lon = east_station['lon'].values[0]


In [ ]:
east_station

In [ ]:
west_station

In [ ]:

print(f"La estación más al oeste está en la latitud {west_lat} y longitud {west_lon}")
print(f"La estación más al este está en la latitud {east_lat} y longitud {east_lon}")



In [ ]:
API_key = 'qZDURhuszLM8Zw-w-ab_KNerRoQ-O-3PBW9GwNOI8yM'
modes =   ['bicycle', 'scooter', 'car']
URLRH_base = 'https://router.hereapi.com/v8/routes?transportMode={Mode}&origin={lat_O},{lon_O}&destination={lat_D},{lon_D}&return=polyline,summary&apikey={APIkey}'

In [ ]:
Colors = ['red', 'blue', 'green']
for i in range(len(modes)):
    URLRH =  URLRH_base.format(Mode = modes[i], lat_O = west_lat, lon_O=west_lon, lat_D=east_lat, lon_D=east_lon, APIkey=API_key)
    rRH = requests.get(URLRH)
    #print('Ruta para: ', modes[i])
    #rRH.json()
    print(URLRH)
    data = rRH.json()
    route = data['routes'][0]['sections'][0]
    duration_hours = route['summary']['duration']/3600
    len_km = route['summary']['length']/1000
    print('El viaje durará ',duration_hours,'horas y el recorrido será de: ', len_km)
    mapa = folium.Map(location = [west_lat, west_lon], zoom_start=14, tiles='CartoDB positron')
    folium.PolyLine(
        locations = fp.decode( route['polyline']), color=Colors[i]
).add_to(mapa)
    mapa.save(f'mapa_{modes[i]}.html')
    print('Mapa guardado como ' f'mapa_{modes[i]}.html')


In [ ]:
# prompt: añade en un solo mapa las 3 rutas con distintos grosores de linea

mapa = folium.Map(location = [west_lat, west_lon], zoom_start=14, tiles='CartoDB positron')
grosores = [1, 10, 3]
for i in range(len(modes)):
  URLRH =  URLRH_base.format(Mode = modes[i], lat_O = west_lat, lon_O=west_lon, lat_D=east_lat, lon_D=east_lon, APIkey=API_key)
  rRH = requests.get(URLRH)
  data = rRH.json()
  route = data['routes'][0]['sections'][0]

  # Define el grosor de la línea según el modo de transporte
  if modes[i] == 'bicycle':
    weight = 3
  elif modes[i] == 'scooter':
    weight = 5
  else:  # car
    weight = 8

  folium.PolyLine(
      locations = fp.decode( route['polyline']), color=Colors[i], weight= grosores[i]
  ).add_to(mapa)

mapa
